In [2]:
import sys
import ase
import nglview as nv
import numpy as np

sys.path.append('../')


In [3]:
from analyses.metrics import *

/Users/songk/miniconda3/envs/sh-net/lib/python3.10/site-packages/flax/struct.py:136: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/Users/songk/miniconda3/envs/sh-net/lib/python3.10/site-packages/flax/struct.py:136: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/Users/songk/miniconda3/envs/sh-net/lib/python3.10/site-packages/flax/struct.py:136: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)


In [13]:
generated_paths = {
    "Symphony": "/Users/songk/atomicarchitects/generated_mols_200k_steps/",
    # silica-allegro-200k-train-steps-all-nov17
}

In [14]:
def get_all_molecules(molecules_dir: str) -> List[Chem.Mol]:
    """Returns all molecules in a directory."""
    molecules = []
    for molecules_file in [f for f in os.listdir(molecules_dir) if f.endswith(".xyz")]:
    # for molecules_file in sorted([f for f in os.listdir(molecules_dir) if f.endswith(".xyz")], key=key_fn):
        if not molecules_file.endswith(".xyz"):
            continue

        molecules_file = os.path.join(molecules_dir, molecules_file)
        mol = xyz_to_rdkit_molecule(molecules_file)
        molecules.append(mol)

    return molecules


all_generated_molecules = {
    model: get_all_molecules(molecules_dir) for model, molecules_dir in generated_paths.items()
}

In [15]:
all_generated_molecules

{'Symphony': [<rdkit.Chem.rdchem.Mol at 0x2caacae80>,
  <rdkit.Chem.rdchem.Mol at 0x2caaddbc0>]}

In [16]:
for model, generated_molecules in all_generated_molecules.items():
    validity_fraction = compute_validity(generated_molecules, get_all_valid_molecules(generated_molecules))
    print(f"{model}: {100 * validity_fraction:0.2f}")

Symphony: 1.56


In [99]:
all_generated_molecules["Symphony (train)"][0]

In [100]:
generated_molecules_with_H_removed = []
true_molecules_with_H_removed = []
for generated_molecule, true_molecule, H_removed in zip(all_generated_molecules["Symphony (train)"], train_mol_list, train_H_removed):
    if H_removed:
        generated_molecules_with_H_removed.append(generated_molecule)
        true_molecules_with_H_removed.append(true_molecule)

print(generated_molecules_with_H_removed[0], true_molecules_with_H_removed[0])
ase_mols = [
    ase.Atoms(
        symbols=[atom.GetSymbol() for atom in generated_molecules_with_H_removed[0].GetAtoms()],
        positions=generated_molecules_with_H_removed[0].GetConformer().GetPositions(),
    ),
    true_molecules_with_H_removed[0]]

nv.show_ase(ase_mols[0])


<rdkit.Chem.rdchem.Mol object at 0x336685df0> Atoms(symbols='C3FN2OCNH3', pbc=False)


NGLWidget()

In [101]:
nv.show_ase(ase_mols[1])

NGLWidget()

In [57]:
invalid_molecules_symphony = [mol for mol in all_generated_molecules["Symphony (train)"][:100] if not check_molecule_validity(mol)]
invalid_molecules_gschnet = [mol for mol in all_generated_molecules["GSchNet (train)"][:100] if not check_molecule_validity(mol)]

In [58]:
invalid_molecules_ase = [
    ase.Atoms(
        symbols=[atom.GetSymbol() for atom in mol.GetAtoms()],
        positions=mol.GetConformer().GetPositions(),
    )
    for mol in invalid_molecules_symphony
]
nv.show_ase(invalid_molecules_ase[2], gui=True)        

NGLWidget()